In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../src")
import os
import torch
import torchvision 
import h5py
import torchvision.transforms as transforms
import numpy as np
import keras

from dataset import get_mean_std

from keras.datasets import mnist, cifar10

Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() #cifar10

In [4]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [6]:
# img_channels, img_rows, img_cols = 1, 28, 28 # mnist
img_channels, img_rows, img_cols = 3, 32, 32 #cifar
X_train = X_train.reshape(X_train.shape[0], img_channels, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], img_channels, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.
X_test /= 255.
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])
X_train = np.vstack((X_train,X_test))
y_true = np.hstack((y_train,y_test))
y_true = y_true.astype('int64')

In [7]:
print(X_train.shape)
print(y_true.shape)
merge_data = X_train
merge_targets = y_true

(60000, 3, 32, 32)
(60000,)


In [84]:
dataset1 = torchvision.datasets.MNIST("../data/", train=True, download=False, 
                                             transform=transforms.Compose([transforms.ToTensor()]))
dataset2 = torchvision.datasets.MNIST("../data/", train=False, download=False, 
                                             transform=transforms.Compose([transforms.ToTensor()]))

In [85]:
data1, targets1 = np.array(dataset1.data, dtype=np.float32), np.array(dataset1.targets)
data2, targets2 = np.array(dataset2.data, dtype=np.float32), np.array(dataset2.targets)

data1 /= 255.0
data2 /= 255.0

merge_data = np.vstack((data1, data2))
merge_targets = np.hstack((targets1, targets2))

In [14]:
merge_data = merge_data.reshape(merge_data.shape[0], -1, merge_data.shape[1], merge_data.shape[2])

print(merge_data.shape)

print(merge_targets.shape)

(60000, 32, 3, 32)
(60000,)


In [8]:
get_mean_std(merge_data, ratio=1)

(array([0.50764906, 0.46386194, 0.45018676], dtype=float32),
 array([0.26785395, 0.24093068, 0.24120948], dtype=float32))

## Create dataset

In [9]:
data_dir = "../data"
dataset_name = "datasets.hdf5"
ab_path = os.path.join(data_dir, dataset_name)

In [10]:
# Image for pytorch Dataset should be (samples, height, width, channels)
merge_data = merge_data.transpose((0, 2, 3, 1))

In [11]:
print(merge_data.shape)

(60000, 32, 32, 3)


In [12]:
with h5py.File(ab_path, "w") as f:
#     grp = f.create_group("mnist")
    grp = f.create_group("cifar10")
    grp.create_dataset("data", merge_data.shape, dtype=merge_data.dtype)
    grp.create_dataset("targets", merge_targets.shape, dtype=merge_targets.dtype)
    grp["data"][:,:,:,:] = merge_data
    grp["targets"][:] = merge_targets